In [77]:
import numpy as np
from scipy import stats as stats
import os
os.environ['MNE_3D_OPTION_ANTIALIAS']='false' # to make visualization work
import imageio.v3 as iio
import pickle

import mne
from mne.epochs import equalize_epoch_counts
from mne.stats import (spatio_temporal_cluster_1samp_test,
                       summarize_clusters_stc)
from mne.minimum_norm import apply_inverse, read_inverse_operator


In [213]:
subjects_dir = 'D:\\Ekaterina_Voevodina\\memory_formation\\data\\subjects'
mris_dir = 'D:\\Ekaterina_Voevodina\\memory_formation\\data\\mri'
src_fname = os.path.join(mris_dir, 'fsaverage', 'bem', 'fsaverage-ico-5-src.fif')
cond1 = 'hits'
cond2 = 'miss'
fband = 'theta'
stcs_dir = os.path.join('D:\\Ekaterina_Voevodina\\memory_formation\\data\\group\\fsaverage_stc', fband)
stc_hits_all, stc_miss_all = [], []

for subject_name in os.listdir(subjects_dir):
    stcs_hits_path = os.path.join(stcs_dir, f'fsaverage_stc_{subject_name}_{cond1}_{fband}-stc.h5')
    stcs_miss_path = os.path.join(stcs_dir, f'fsaverage_stc_{subject_name}_{cond2}_{fband}-stc.h5')
    stc_hits = mne.read_source_estimate(stcs_hits_path)
    stc_miss = mne.read_source_estimate(stcs_miss_path)
    stc_hits_all.append(stc_hits.data)
    stc_miss_all.append(stc_miss.data)


In [32]:
len(stc_hits_all)

19

In [71]:
stc_hits

<SourceEstimate | 20484 vertices, subject : fsaverage, tmin : -1500.0 (ms), tmax : 3995.0 (ms), tstep : 5.0 (ms), data shape : (20484, 1100), ~172.1 MB>

In [33]:
for stc in stc_hits_all:
    print(stc.shape)

(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)
(20484, 1100)


In [56]:
X = np.stack([np.stack(stc_hits_all, -1),
np.stack(stc_miss_all, -1)], -1)

In [208]:
src = mne.read_source_spaces(src_fname)
adjacency = mne.spatial_src_adjacency(src)
fsave_vertices = [s['vertno'] for s in src]

    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
-- number of adjacent vertices : 20484


In [51]:
X.shape

(20484, 1100, 19, 2)

In [57]:
X = np.abs(X)  # only magnitude
X = X[:, :, :, 0] - X[:, :, :, 1]  # make paired contrast

In [58]:
X = np.transpose(X, [2, 1, 0])

In [59]:
p_threshold = 0.001
df = len(os.listdir(subjects_dir)) - 1  # degrees of freedom for the test
t_threshold = stats.distributions.t.ppf(1 - p_threshold / 2, df=df)

# Now let's actually do the clustering. This can take a long time...
print('Clustering.')
T_obs, clusters, cluster_p_values, H0 = clu = \
    spatio_temporal_cluster_1samp_test(X, adjacency=adjacency, n_jobs=None,
                                       threshold=t_threshold, buffer_size=None,
                                       verbose=True)

Clustering.
stat_fun(H1): min=-8.541796 max=5.935616
Running initial clustering …
Found 2558 clusters


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

In [143]:
type(cluster_p_values)

numpy.ndarray

In [ ]:
# SAVING THE DATA
# save np.arrays
path_to_save = 'D:\\Ekaterina_Voevodina\\memory_formation\\data\\group\\statistics'
X_path_to_save = os.path.join(path_to_save, f'X_{cond1}_VS_{cond2}_{fband}')
T_obs_path_to_save = os.path.join(path_to_save, f'T_obs_{cond1}_VS_{cond2}_{fband}')
cluster_p_values_path_to_save = os.path.join(path_to_save, f'cluster_p_values_{cond1}_VS_{cond2}_{fband}')
H0_path_to_save = os.path.join(path_to_save, f'H0_{cond1}_VS_{cond2}_{fband}')

np.save(X_path_to_save, X)
np.save(T_obs_path_to_save, T_obs)
np.save(cluster_p_values_path_to_save, cluster_p_values)
np.save(H0_path_to_save, H0)

# save lists
# save stc_hits_all
with open(os.path.join(path_to_save, f'stc_hits_all_{cond1}_VS_{cond2}_{fband}'), "wb") as fp:   #Pickling
    pickle.dump(stc_hits_all, fp)
    
# save stc_miss_all
with open(os.path.join(path_to_save, f'stc_miss_all_{cond1}_VS_{cond2}_{fband}'), "wb") as fp:   #Pickling
    pickle.dump(stc_miss_all, fp)

# save clusters
with open(os.path.join(path_to_save, f'clusters_{cond1}_VS_{cond2}_{fband}'), "wb") as fp:   #Pickling
    pickle.dump(clusters, fp)

# save clu
with open(os.path.join(path_to_save, f'clu_{cond1}_VS_{cond2}_{fband}'), "wb") as fp:   #Pickling
    pickle.dump(clu, fp)

# save fsave_vertices
with open(os.path.join(path_to_save, 'fsave_vertices'), "wb") as fp:   #Pickling
    pickle.dump(fsave_vertices, fp)


In [ ]:
# LOADING saved files
X_file = np.load(X_path_to_save + '.npy')
T_obs_file = np.load(T_obs_path_to_save + '.npy')
cluster_p_values_file = np.load(cluster_p_values_path_to_save + '.npy')
H0_file = np.load(H0_path_to_save + '.npy')

# load stc_hits_all
with open(os.path.join(path_to_save, f'stc_hits_all_{cond1}_VS_{cond2}_{fband}'), "rb") as fp:   # Unpickling
    stc_hits_all_file = pickle.load(fp)

# load stc_miss_all
with open(os.path.join(path_to_save, f'stc_miss_all_{cond1}_VS_{cond2}_{fband}'), "rb") as fp:   # Unpickling
    stc_miss_all_file = pickle.load(fp)

# load clusters
with open(os.path.join(path_to_save, f'clusters_{cond1}_VS_{cond2}_{fband}'), "rb") as fp:   # Unpickling
    clusters_file = pickle.load(fp)

# load clu
with open(os.path.join(path_to_save, f'clu_{cond1}_VS_{cond2}_{fband}'), "rb") as fp:   # Unpickling
    clu_file = pickle.load(fp)

# load fsave_vertices
with open(os.path.join(path_to_save, 'fsave_vertices'), "rb") as fp:   #Pickling
    fsave_vertices_file = pickle.load(fp)

In [186]:
# cheching for equality: information before and after saving
#! DO NOT INCLUDE IN FINAL SCRIPT

for i in range(len(clusters_file)):
    if np.array_equal(clusters_file[i], clusters[i]):
        continue
    else:
        print(f'{i} NOT ok')

In [60]:
# Select the clusters that are statistically significant at p < 0.05
good_clusters_idx = np.where(cluster_p_values < 0.05)[0]
good_clusters = [clusters[idx] for idx in good_clusters_idx]

In [61]:
good_clusters_idx

array([244, 253, 436, 521, 681, 836], dtype=int64)

In [204]:
type(fsave_vertices[0])

numpy.ndarray

In [102]:
# # trying to make visualization work
#! DOES NOT HELP :(

# mne.viz.set_3d_options(depth_peeling=False, antialias=True, multi_samples=1)
# mne.viz.set_3d_options(antialias=False, depth_peeling=True, smooth_shading=False, multi_samples=1)

# # OR

# os.environ['MNE_3D_OPTION_ANTIALIAS']='false'
# os.environ['MNE_3D_OPTION_DEPTH_PEELING']='false'
# os.environ['MNE_3D_OPTION_SMOOTH_SHADING']='false'

In [210]:
stc_all_cluster_vis = summarize_clusters_stc(clu_file, #or clu
                                             vertices=fsave_vertices_file, #or fsave_vertices
                                             subject='fsaverage')

# blue blobs are for condition A < condition B, red for A > B
brain = stc_all_cluster_vis.plot(
    hemi='both', views='lateral', subjects_dir=mris_dir,
    time_label='temporal extent (ms)', size=(800, 800),
    smoothing_steps=5, clim=dict(kind='value', pos_lims=[0, 1, 40]))

In [79]:
type(stc_all_cluster_vis)

mne.source_estimate.SourceEstimate

In [81]:
len(stc_all_cluster_vis.data > 0)

20484